In [ ]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
REPO = "https://github.com/<your-username>/<your-repo>.git"  # TODO: replace with your repo
!git clone $REPO

import os, re
repo_name = re.sub(r".*/(.*)\\.git$", r"\\1", REPO)
%cd $repo_name

!pip -q install -r requirements.txt

In [ ]:
# Create data/ and upload anime.csv, rating.csv
import os
os.makedirs("data", exist_ok=True)

from google.colab import files
print("Upload anime.csv and rating.csv")
uploaded = files.upload()
for name in uploaded.keys():
    if name.endswith(".csv"):
        !mv $name data/
!ls -lh data || true